In [5]:
import json

In [6]:
with open("transactions.json") as f:
    data_copy = json.load(f)
data_copy

[{'Date': '2018-09-21 00:00:00',
  'Account': 'Card',
  'Category': 'Other',
  'Note': 'Bank',
  'Amount': '4262',
  'Type': 'Income'},
 {'Date': '2018-09-21 00:00:00',
  'Account': 'Card',
  'Category': 'Other',
  'Note': 'Start Amount',
  'Amount': '17403',
  'Type': 'Income'},
 {'Date': '2018-09-21 00:00:00',
  'Account': 'Nordnet',
  'Category': 'Other',
  'Note': 'Aktier',
  'Amount': '10345',
  'Type': 'Income'},
 {'Date': '2018-09-23 00:00:00',
  'Account': 'Card',
  'Category': 'Food',
  'Note': 'mcdonalds',
  'Amount': '44',
  'Type': 'Expense'},
 {'Date': '2018-09-25 00:00:00',
  'Account': 'Nordnet',
  'Category': 'Investment',
  'Note': 'aktier uppgång',
  'Amount': '25',
  'Type': 'Income'},
 {'Date': '2018-09-25 00:00:00',
  'Account': 'Nordnet',
  'Category': 'Investment',
  'Note': 'aktier nedgång',
  'Amount': '-6',
  'Type': 'Income'},
 {'Date': '2018-09-25 00:00:00',
  'Account': 'Nordnet',
  'Category': 'Investment',
  'Note': 'aktier uppgång 21-28 augusti 2018',
  

In [8]:
transactions = data_copy

In [12]:
from datetime import datetime

def transform_transaction(tx: dict) -> dict:
    # Convert the date string to a datetime object.
    dt = datetime.strptime(tx["Date"], "%Y-%m-%d %H:%M:%S")
    # Convert amount to a float and get the absolute value.
    amount = float(tx["Amount"])
    abs_amount = abs(amount)
    
    # Check if this is a transfer transaction.
    if tx["Type"].lower() == "transfer-out":
        # Map fields for a TransferTransaction.
        return {
            "dateTime": dt.isoformat(),
            "note": tx["Note"],
            "amount": abs_amount,
            "type": "transfer",
            "fromAccount": tx["Account"],
            "toAccount": tx["Category"]
        }
    else:
        # For DebitCreditTransaction, determine the new type.
        # Income: positive -> credit, negative -> debit.
        # Expense: always debit.
        if tx["Type"].lower() == "income":
            new_type = "credit" if amount >= 0 else "debit"
        elif tx["Type"].lower() == "expense":
            new_type = "debit"
        else:
            # Fallback: you can adjust this as needed.
            new_type = "debit"
        
        return {
            "dateTime": dt.isoformat(),
            "note": tx["Note"],
            "amount": abs_amount,
            "type": new_type,
            "account": tx["Account"],
            "category": tx["Category"],
            "tag": []  # Empty list for tags
        }

In [13]:
new_transactions = [transform_transaction(tx) for tx in transactions]


In [14]:
with open("transactions_transformed.json", "w") as f:
    json.dump(new_transactions, f, indent=2)